## Imports

In [ ]:
import numpy as np
from sklearn.metrics import DistanceMetric

import xml.etree.ElementTree as et
from plotly.offline import iplot, plot, init_notebook_mode
import plotly.graph_objs as go

init_notebook_mode()

In [ ]:
namespaces = [
    "http://www.topografix.com/GPX/1/1 http://www.topografix.com/GPX/1/1/gpx.xsd http://www.garmin.com/xmlschemas/GpxExtensions/v3 http://www.garmin.com/xmlschemas/GpxExtensionsv3.xsd http://www.garmin.com/xmlschemas/TrackPointExtension/v1 http://www.garmin.com/xmlschemas/TrackPointExtensionv1.xsd http://www.garmin.com/xmlschemas/GpxExtensions/v3 http://www.garmin.com/xmlschemas/GpxExtensionsv3.xsd http://www.garmin.com/xmlschemas/TrackPointExtension/v1 http://www.garmin.com/xmlschemas/TrackPointExtensionv1.xsd http://www.garmin.com/xmlschemas/GpxExtensions/v3 http://www.garmin.com/xmlschemas/GpxExtensionsv3.xsd http://www.garmin.com/xmlschemas/TrackPointExtension/v1 http://www.garmin.com/xmlschemas/TrackPointExtensionv1.xsd http://www.garmin.com/xmlschemas/GpxExtensions/v3 http://www.garmin.com/xmlschemas/GpxExtensionsv3.xsd http://www.garmin.com/xmlschemas/TrackPointExtension/v1 http://www.garmin.com/xmlschemas/TrackPointExtensionv1.xsd http://www.garmin.com/xmlschemas/GpxExtensions/v3 http://www.garmin.com/xmlschemas/GpxExtensionsv3.xsd http://www.garmin.com/xmlschemas/TrackPointExtension/v1 http://www.garmin.com/xmlschemas/TrackPointExtensionv1.xsd http://www.garmin.com/xmlschemas/GpxExtensions/v3 http://www.garmin.com/xmlschemas/GpxExtensionsv3.xsd http://www.garmin.com/xmlschemas/TrackPointExtension/v1 http://www.garmin.com/xmlschemas/TrackPointExtensionv1.xsd http://www.garmin.com/xmlschemas/GpxExtensions/v3 http://www.garmin.com/xmlschemas/GpxExtensionsv3.xsd http://www.garmin.com/xmlschemas/TrackPointExtension/v1 http://www.garmin.com/xmlschemas/TrackPointExtensionv1.xsd",
    "http://www.w3.org/2001/XMLSchema-instance",
    "http://www.topografix.com/GPX/1/1",
]

for namespace in namespaces:
    et.register_namespace("", namespace)

tree = et.parse("activity_data/250704_super_bad_gps.gpx")
root = tree.getroot()

temp = [
    [float(n.attrib["lat"]) * np.pi / 180, float(n.attrib["lon"]) * np.pi / 180] for n in root[1][2]
]
pts = np.array(temp)

dist = DistanceMetric.get_metric("haversine")

ref = pts[0]

ref

In [ ]:
pts[:4,]

In [ ]:
print(root)

In [ ]:
epsilon = 0.00001
epsilon_lat = ref + [epsilon, 0]
epsilon_lon = ref + [0, epsilon]

add_lat = np.array([ref, epsilon_lat])
add_lon = np.array([ref, epsilon_lon])
print(dist.pairwise(add_lat))
print(dist.pairwise(add_lon))

corr_factor = dist.pairwise(add_lat)[0, 1] / dist.pairwise(add_lon)[0, 1]
print(corr_factor)

In [ ]:
window = 7
smoothed = 1 * pts[: -window + 1]

for i in range(1, window):
    print(f"{i = }")
    if i != window - 1:
        smoothed += 1 * pts[i : -window + i + 1]
    elif i == window - 1:
        smoothed += pts[i:]

smoothed /= window
# smoothed = (
#     1 * pts[:-8] +
#     1 * pts[1:-7] +
#     2 * pts[2:-6] +
#     3 * pts[3:-5] +
#     4 * pts[4:-4] +
#     3 * pts[5:-3] +
#     2 * pts[6:-2] +
#     1 * pts[7:-1] +
#     1 * pts[8:]
# ) / 18
# smoothed = (1*pts[:-6] + 1*pts[1:-5] + 2*pts[2:-4] + 3*pts[3:-3] + 2*pts[4:-2] + 1*pts[5:-1] + 1*pts[6:]) / 11
# smoothed = (1*pts[:-4] + 1.5*pts[1:-3] + 4*pts[2:-2] + 1.5*pts[3:-1] + 1*pts[4:]) / 9
# smoothed = (1*pts[:-4] + 2*pts[1:-3] + 3*pts[2:-2] + 2*pts[3:-1] + 1*pts[4:]) / 9

In [ ]:
trace = go.Scatter(
    x=pts[:, 1],
    y=pts[:, 0] * corr_factor,
)

trace_smoothed = go.Scatter(
    x=smoothed[:, 1],
    y=smoothed[:, 0] * corr_factor,
)

data = [trace, trace_smoothed]

layout = go.Layout(
    yaxis=dict(scaleanchor="x", scaleratio=1),
    height=600,
)

iplot(go.Figure(data=data, layout=layout))

# Delete initial and final nodes

In [ ]:
node = root[1][2]

In [ ]:
# for index in [-1,-1,-1, 0, 0, 0]:
# # for index in [-1,-1, 0, 0]:
#     print(node[index].attrib)
#     node.remove(node[index])

# Delete elevation nodes

In [ ]:
offset = window / 3

for i in range(len(node)):
    node[i].remove(node[i][0])

# Replace lats and lons

In [ ]:
assert len(node) == len(smoothed)

In [ ]:
print(f"{len(smoothed) = }")
print(f"{len(node) = }")

In [ ]:
for i in range(3, len(smoothed)):
    node[i].attrib["lat"] = "{:.7f}".format(smoothed[i, 0] * 180 / np.pi)
    node[i].attrib["lon"] = "{:.7f}".format(smoothed[i, 1] * 180 / np.pi)

# Write out (and don't forget to fix the namespace BS after

In [ ]:
tree.write("fixed_for_upload.gpx")

- find and replace `ns2` with `gpxtpx` everywhere
- replace schema namespace bullshit on L1 with same front matter from original file